In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import cufflinks

In [24]:
dt=0.01
vel=[]
pos=[]
v=-4
x=0
g=10
k=15
m=37.5
N=1000
for i in range(N):
    # print(v)
    a=g-k*v**2/m
    v+=a*dt
    x+=v*dt
    vel.append(v)
    pos.append(x)
pd.Series(vel).iplot()

In [25]:
t=2*(np.linspace(0,N*dt,N)-0.55)
xp=5*np.tanh(t)
pd.Series(xp).iplot()

In [26]:
time=pd.Series([
0,
9.85977,
19.9387,
30.0175,
39.6582,
44.6976,
49.7371,
60.0351,
70.1139,
79.7546,
89.8335,
99.9124,
109.991,
120.289,
129.93,
140.009,
150.088,
160.386,
170.245,
180.324,
190.403,
200.482,
210.561,
220.421,
230.28,
240.14,
250,
260.079,
])


height_r=pd.Series(
    [
39.0506,
38.6051,
36.8304,
34.6127,
31.4457,
29.6089,
27.7722,
24.0354,
21.3747,
19.0937,
17.2557,
15.6709,
14.4025,
13.1974,
12.119,
11.1671,
10.2785,
9.26319,
8.43791,
7.61258,
6.85053,
6.02519,
5.51631,
4.88091,
4.37208,
3.48351,
3.16456,
2.52909,

    ]
)
fig=go.Figure()
fig.add_trace(go.Scatter(x=time,y=height_r,name=f'理论计算'))

In [27]:
time=pd.Series([0,
9.91282,
19.825,
30.3646,
40.6698,
50.5062,
60.4788,
70.2028,
80.3866,
90.3765,
99.9355,
110.152,
120.37,
130.603,
140.193,
150.21,
160.22,
170.448,
180.043,
190.06,
200.708,
210.087,
220.108,
230.342,
240.148,
250.168,
260.395
])

vel=pd.Series(
    [
        0,
98.2226,
195.843,
282.019,
348.315,
374.851,
329.098,
250.211,
202.652,
173.164,
139.458,
122.621,
106.989,
105.213,
101.025,
96.838,
86.6272,
80.6339,
80.0604,
75.8737,
63.8576,
61.4762,
61.5064,
60.9348,
58.5547,
57.9824,
50.7843,

    ]
)
fig=go.Figure()
fig.add_trace(go.Scatter(x=time[:6],y=vel[:6],name=f'理论计算'))

In [43]:
t_max=260
N=1000
dt=t_max/N
vel_s=[]
height=[]
a_list=[]
P_list=[]
dP_list=[]
sos_list=[] #list of sound in an specific height
# h=39.0506e3
h=40e3
g=9.6
G=6.67e-11
m=190
M=5.965e24
R=6371.393e3
gamma=1.4 # ratio of the molar heat capacity

v=0

def get_T(h):
    h/=1e3
    h_p=6357.766*h/(6357.766+h)
    # print(h_p)
    if(0<=h_p<=11):
        return 288.15 - 6.5*h_p
    elif(11<h_p<=20):
        return 216.65
    elif(20<h_p<=32):
        return 216.65+(h_p-20)
    elif(32<h_p<=47):
        return 228.65 + 2.8*(h_p-32)
    elif(47<h_p<=51):
        return 270.65 
    elif(51<h_p<=71):
        return 270.65 - 2.8 *(h_p-51)
    elif( 71 <h_p <= 84.852):
        return 214.65 - 2.0 *(h_p-71) 
    elif(86 <= h):
        return 186.8673 

    
get_v_s=lambda h:np.sqrt(1.4*8.314*get_T(h)/0.029)

def get_P(h):
    h/=1e3
    h_p=6357.766*h/(6357.766+h)
    if 0 <= h_p <= 11:
        pressure = 1013.25 * (288.15 / (288.15 - 6.5 * h_p)) ** (-34.1632 / 6.5)
    elif 11 < h_p <= 20:
        pressure = 226.3226 * np.exp(-34.1632 * (h_p - 11) / 216.65)
    elif 20 < h_p <= 32:
        pressure = 54.74980 * (216.65 / (216.65 + (h_p - 20))) ** 34.1632
    elif 32 < h_p <= 47:
        pressure = 8.680422 * (228.65 / (228.65 + 2.8 * (h_p - 32))) ** (34.1632 / 2.8)
    else:
        return 0
    
    return pressure

sos_list.append(get_v_s(h))
last_P=0
# c=
for i in range(N):
    if(h<0):
        break
    # print(v)
    # d=1.225*np.e**(-h/8882)
    d=1.5*np.e**(-h/6450)
    a=G*M/(h+R)**2-0.003747388815148012*d*v**2
    a_list.append(a)
    v+=a*dt
    h-=v*dt
    
    sos=get_v_s(h) #speed of sound
    Mach=v/sos # Mach Number
    P0=get_P(h)
    P=(2*gamma*Mach**2-(gamma-1))*P0/(gamma+1)
    P_list.append(P0*10)
    dP=(P-last_P)/dt
    last_P=P
    dP_list.append(dP)
    sos_list.append(sos)
    vel_s.append(v)
    height.append(h)
# print(list(map(get_T,height)))
fig=go.Figure()
time_list=np.linspace(0,t_max,N)
# fig.add_trace(go.Scatter(x=time[5:],y=vel[5:],name=f'recorded data'))
# fig.add_trace(go.Scatter(x=np.linspace(time[5],t_max,N),y=vel_s,name=f'simulation'))
# fig.add_trace(go.Scatter(x=time,y=vel,name=f'recorded data'))
fig.add_trace(go.Scatter(x=np.array(height),y=vel_s,name=f'simulation'))
fig.add_trace(go.Scatter(x=np.array(height),y=sos_list,name=f'speed of sound'))
# fig.add_trace(go.Scatter(x=time_list,y=np.array(height)/10,name=f'height'))

In [29]:
h=91
h_p=6357.766*h/(6357.766+h)
h_p

89.715878355642

In [30]:
h0=2.53e3
m=190
d=1.5*np.e**(-h0/6450)
v=50.784
G*M*m/((R+h0)**2*d*v**2)

0.7120038748781223

In [31]:
pd.Series(a_list).iplot()  # acceleration

In [32]:
pd.Series(P_list).iplot()  # pressure on body

In [33]:
pd.Series(dP_list[1:]).iplot()  # pressure on body

In [34]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=time,y=height_r*1e3,name=f'recorded data'))
fig.add_trace(go.Scatter(x=np.linspace(0,t_max,N),y=height,name=f'simulation'))

In [35]:
from scipy.optimize import curve_fit
a_f=lambda x,a:a*x
popt, pcov=curve_fit(a_f, time[:6],vel[:6])
print(popt)
fit_time = np.linspace(0, time[5], N)  # 用于拟合的时间点
fit_vel = popt[0]*fit_time

fig=go.Figure()
fig.add_trace(go.Scatter(x=time[:6],y=vel[:6],name=f'recorded data'))
fig.add_trace(go.Scatter(x=fit_time,y=fit_vel,name=f'simulation'))

[8.27983031]


In [36]:
def get_max_vel(h):
    t_max=260
    N=1000
    dt=t_max/N
    vel_s=[]
    height=[]
    h=h*1e3  #km
    g=9.6
    v=0
    for i in range(N):
        # print(v)
        # d=1.225*np.e**(-h/8882)
        d=1.5*np.e**(-h/6450)
        a=g-g*d*v**2/(122**2)
        v+=a*dt
        h-=v*dt
        vel_s.append(v)
        height.append(h)
    return max(vel_s)
h=np.linspace(10,40,60)
max_vel_list=list(map(get_max_vel,list(h)))
fig=go.Figure()
fig.add_trace(go.Scatter(x=h,y=max_vel_list,name=f'recorded data'))
# get_max_vel(27)

In [37]:
def get_max_acc(h):
    vel_s=[]
    height=[]
    a_list=[]
    P_list=[]
    dP_list=[]
    sos_list=[] #list of sound in an specific height
    g=9.6
    G=6.67e-11
    m=190
    M=5.965e24
    R=6371.393e3
    gamma=1.4 # ratio of the molar heat capacity

    v=0
    last_P=0
    for i in range(N):
        if(h<0):
            break
        # print(v)
        # d=1.225*np.e**(-h/8882)
        d=1.5*np.e**(-h/6450)
        a=G*M/(h+R)**2-0.003747388815148012*d*v**2
        a_list.append(a)
        v+=a*dt
        h-=v*dt
        
        sos=get_v_s(h) #speed of sound
        Mach=v/sos # Mach Number
        P0=get_P(h)
        P=(2*gamma*Mach**2-(gamma-1))*P0/(gamma+1)
        P_list.append(P0*10)
        dP=(P-last_P)/dt
        last_P=P
        dP_list.append(dP)
        sos_list.append(sos)
        vel_s.append(v)
        height.append(h)
    return max(np.abs(a_list))
get_max_acc(40e3)

9.679022167584693

In [38]:
import numpy as np
import matplotlib.pyplot as plt
N=1000
dt=300/N

def get_max_acc(h):
    vel_s=[]
    height=[]
    a_list=[]
    P_list=[]
    dP_list=[]
    sos_list=[] #list of sound in an specific height
    g=9.6
    G=6.67e-11
    m=190
    M=5.965e24
    R=6371.393e3
    gamma=1.4 # ratio of the molar heat capacity

    v=0
    last_P=0
    for i in range(N):
        if(h<0):
            break
        # print(v)
        # d=1.225*np.e**(-h/8882)
        d=1.5*np.e**(-h/6450)
        a=G*M/(h+R)**2-0.003747388815148012*d*v**2
        a_list.append(a)
        v+=a*dt
        h-=v*dt
        
        vel_s.append(v)
        height.append(h)
    return max(np.abs(a_list))
H=np.linspace(60,200,100)
acc_list=[get_max_acc(h*1e3) for h in H]

fig=go.Figure()
fig.add_trace(go.Scatter(x=H,y=acc_list,name=f'recorded data'))

In [41]:
get_P(22.6e3)

36.88113714995549

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Data of time and height
time1 = pd.Series([
    0, 9.85977, 19.9387, 30.0175, 39.6582, 44.6976, 49.7371, 60.0351, 70.1139, 79.7546,
    89.8335, 99.9124, 109.991, 120.289, 129.93, 140.009, 150.088, 160.386, 170.245,
    180.324, 190.403, 200.482, 210.561, 220.421, 230.28, 240.14, 250, 260.079
])

height0 = pd.Series([
    39.0506, 38.6051, 36.8304, 34.6127, 31.4457, 29.6089, 27.7722, 24.0354, 21.3747,
    19.0937, 17.2557, 15.6709, 14.4025, 13.1974, 12.119, 11.1671, 10.2785, 9.26319,
    8.43791, 7.61258, 6.85053, 6.02519, 5.51631, 4.88091, 4.37208, 3.48351, 3.16456,
    2.52909
])

# Data of time and velocity
time2 = pd.Series([
    0, 9.91282, 19.825, 30.3646, 40.6698, 50.5062, 60.4788, 70.2028, 80.3866, 90.3765,
    99.9355, 110.152, 120.37, 130.603, 140.193, 150.21, 160.22, 170.448, 180.043,
    190.06, 200.708, 210.087, 220.108, 230.342, 240.148, 250.168, 260.395
])

velocity = pd.Series([
    0, 98.2226, 195.843, 282.019, 348.315, 374.851, 329.098, 250.211, 202.652, 173.164,
    139.458, 122.621, 106.989, 105.213, 101.025, 96.838, 86.6272, 80.6339, 80.0604,
    75.8737, 63.8576, 61.4762, 61.5064, 60.9348, 58.5547, 57.9824, 50.7843
])

R=6371.393e3 # radius of the earth
gamma=1.4 # ratio of the molar heat capacity

v=0

def get_T(h):
    h/=1e3
    h_p=6357.766*h/(6357.766+h)
    if(0<=h_p<=11):
        return 288.15 - 6.5*h_p
    elif(11<h_p<=20):
        return 216.65
    elif(20<h_p<=32):
        return 216.65+(h_p-20)
    elif(32<h_p<=47):
        return 228.65 + 2.8*(h_p-32)
    elif(47<h_p<=51):
        return 270.65 
    elif(51<h_p<=71):
        return 270.65 - 2.8 *(h_p-51)
    elif( 71 <h_p <= 84.852):
        return 214.65 - 2.0 *(h_p-71) 
    elif(86 <= h <= 91):
        return 186.8673
    elif(91 < h <= 100):
        return 263.1905 - 76.3232 * (1 - ((h - 91) / 19.9429) ** 2) ** 0.5
    else:
        return 195.07
get_v_s=lambda h:np.sqrt(1.4*8.314*get_T(h)/0.029)

# pressure in hPa

def get_P(h):
    h/=1e3
    h_p=6357.766*h/(6357.766+h)
    if 0 <= h_p <= 11:
        return 1013.25 * (288.15 / (288.15 - 6.5 * h_p)) ** (-34.1632 / 6.5)
    elif 11 < h_p <= 20:
        return 226.3226 * np.exp(-34.1632 * (h_p - 11) / 216.65)
    elif 20 < h_p <= 32:
        return 54.74980 * (216.65 / (216.65 + (h_p - 20))) ** 34.1632
    elif 32 < h_p <= 47:
        return 8.680422 * (228.65 / (228.65 + 2.8 * (h_p - 32))) ** (34.1632 / 2.8)
    elif 47 < h_p <= 51:
        return 1.109106 * np.exp(-34.1632 * (h_p - 47) / 270.65)
    elif 51 < h_p <= 71:
        return 0.6694167 * (270.65 / (270.65 - 2.8 * (h_p - 51))) ** (-34.1632 / 2.8)
    elif 71 < h_p <= 84.852:
        return 0.03956649 * (214.65 / (214.65 - 2.0 * (h_p - 71))) ** (-34.1632 / 2.0)
    elif 86 < h :
        return np.exp(95.571 - 4.012 * h)
    else: return 1013

# t_max=260

N=10000

# initial height and simulation time

h=39.0506e3
t_max=260

dt=t_max/N
g=9.6
G=6.67e-11
m=190
M=5.965e24

def get_res():
    vel_s=[] # list of simulated velocity
    height=[]
    a_list=[]
    P_list=[] #list of pressure
    sos_list=[] #list of sound in an specific height
    global h,v
    for i in range(N):
        
        if(h<0):
            break
        else:
            d=1.5*np.e**(-h/6450) # density
            a=G*M/(h+R)**2-0.712*d*v**2/m # acceleration
            a_list.append(a) 
            v+=a*dt
            h-=v*dt
            
            sos=get_v_s(h) #speed of sound
            Mach=v/sos # Mach Number
            P0=get_P(h) 
            P=(2*gamma*Mach**2-(gamma-1))*P0/(gamma+1)
            P_list.append(P*100) # hPa to Pa
            sos_list.append(sos)
            
            vel_s.append(v)
            height.append(h/1e3)
    return vel_s,height,a_list,P_list,sos_list

